In [1]:
import os
import PIL
from cleverhans.attacks import FastGradientMethod
import numpy as np
import pandas as pd
from PIL import Image
from scipy.misc import imread
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import randrange
%matplotlib inline
from tensorflow.contrib.slim.nets import inception
import keras
from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
slim = tf.contrib.slim
tf.flags.DEFINE_string(
    'master', '', 'The address of the TensorFlow master to use.')
tf.flags.DEFINE_string(
    'checkpoint_path', 'nips-2017-adversarial-learning-development-set/inception_v3.ckpt', 'Path to checkpoint for inception network.')
tf.flags.DEFINE_string(
    'input_dir', 'nips-2017-adversarial-learning-development-set/images/', 'Input directory with images.')
tf.flags.DEFINE_string(
    'output_dir', '', 'Output directory with images.')
tf.flags.DEFINE_float(
    'max_epsilon', 4.0, 'Maximum size of adversarial perturbation.')
tf.flags.DEFINE_integer(
    'image_width', 299, 'Width of each input images.')
tf.flags.DEFINE_integer(
    'image_height', 299, 'Height of each input images.')
tf.flags.DEFINE_integer(
    'batch_size', 16, 'How many images process at one time.')
FLAGS = tf.flags.FLAGS

In [3]:
def noisy(noise_typ,image):
    #Gaussian
   if noise_typ == 1:
      row,col,ch= image.shape
      mean = 0
      var = 0.1
      sigma = var**0.5
      gauss = np.random.normal(mean,sigma,(row,col,ch))
      gauss = gauss.reshape(row,col,ch)
      noisy = image + gauss
      return noisy.astype('uint8')
    #Salt and Pepper
   elif noise_typ == 2:
      row,col,ch = image.shape
      s_vs_p = 0.5
      amount = 0.004
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 1
      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out
    #Poisson
   elif noise_typ == 3:
      vals = len(np.unique(image))
      vals = 2 ** np.ceil(np.log2(vals))
      noisy = np.random.poisson(image * vals) / float(vals)
      return noisy
    #speckle
   elif noise_typ ==4:
      row,col,ch = image.shape
      gauss = np.random.randn(row,col,ch)
      gauss = gauss.reshape(row,col,ch)        
      noisy = image + image * gauss
      return noisy.astype('uint8')

In [4]:
def subsample(training_images,training_labels, ratio=0.8):
    shuffle_index = np.random.permutation(len(training_labels))
    training_images = training_images[shuffle_index]
    training_labels = training_labels[shuffle_index]
    sample = list()
    sample_labels = list()
    n_sample = round(training_images.shape[0] * ratio)
    while len(sample) < n_sample:
        index = randrange(training_images.shape[0])
        sample.append(training_images[index,:,:,:])
        sample_labels.append(training_labels[index])
    return np.asarray(sample),np.asarray(sample_labels)

In [5]:
def load_training_images(training_image_dir):

    image_index = 0
    
    images = np.ndarray(shape=(500*200, 64,64,3))
    names = []
    labels = []                       
    
    # Loop through all the types directories
    for type in os.listdir(training_image_dir):
        if os.path.isdir(training_image_dir + type + '/images/'):
            type_images = os.listdir(training_image_dir + type + '/images/')
            # Loop through all the images of a type directory
            #batch_index = 0;
            #print ("Loading Class ", type)
            for image in type_images:
                image_file = os.path.join(training_image_dir, type + '/images/', image)

                # reading the images as they are; no normalization, no color editing
                image_data = mpimg.imread(image_file) 
                #print ('Loaded Image', image_file, image_data.shape)
                if (image_data.shape == (64, 64, 3)):
                    images[image_index, :,:,:] = image_data
                    
                    labels.append(type)
                    names.append(image)
                    
                    image_index += 1
                    #batch_index += 1
                #if (batch_index >= batch_size):
                 #   break;
    labels = np.asarray(labels)
    names = np.asarray(names)
    return (images[0:len(labels)].astype('uint8'), labels, names)

In [6]:
def rescale(img):
    img = Image.fromarray(img)
    basewidth =299
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
    return img

In [7]:
imgs,labels,names = load_training_images('tiny-imagenet-200/train/')

In [8]:
imgs = imgs[0:100]
labels = labels[0:100]
names = names[0:100]
imgs_large = np.ndarray(shape= [imgs.shape[0],299,299,3])
for i in range(imgs.shape[0]):
    imgs_large[i,:,:,:] = rescale(imgs[i])
imgs_large=imgs_large.astype('uint8')
imgs_noisy = np.ndarray(shape= imgs_large.shape)
for i in range(imgs_large.shape[0]):
    imgs_noisy[i,:,:,:] = noisy(1,imgs_large[i])
imgs_noisy=imgs_noisy.astype('uint8')
sub_imgs,sub_labels = subsample(imgs_noisy,labels)
batch_shape = [20, 299, 299, 3]
num_classes = 200

In [9]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [10]:
def add_new_last_layer(base_model, nb_classes):
  """Add last layer to the convnet
  Args:
    base_model: keras model excluding top
    nb_classes: # of classes
  Returns:
    new keras model with last layer
  """
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x) 
  predictions = Dense(nb_classes, activation='softmax')(x) 
  model = Model(input=base_model.input, output=predictions)
  return model

In [11]:
def setup_to_finetune(model):
   """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top 
      layers.
   note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in 
         the inceptionv3 architecture
   Args:
     model: keras model
   """
   for layer in model.layers:
      layer.trainable = True
   model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),   
                 loss='categorical_crossentropy')

In [12]:
model = add_new_last_layer(base_model, 201)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [13]:
setup_to_finetune(model)

In [14]:
all_labels = []
for type in os.listdir('tiny-imagenet-200/train/'):
    all_labels = all_labels+[type]

In [15]:
for i in sub_labels:
    ind = all_labels.index(i)
    ind_sub = list(sub_labels).index(i)
    sub_labels[ind_sub] = ind

In [16]:
sub_labels = keras.utils.to_categorical(sub_labels, num_classes=201)

In [17]:
history = model.fit(
    sub_imgs,sub_labels,
    epochs=1,
    batch_size=20
    )
model.save(args.output_model_file)

Epoch 1/1
80/80 [==============================] - 87s 1s/step - loss: 5.8231


NameError: name 'args' is not defined